In [2]:
import os
import sys
sys.path.insert(0, os.path.abspath('../../'))

import numpy as np
import arrayfire as af
from matplotlib import pyplot as plt

from dg_maxwell import params
from dg_maxwell import msh_parser
from dg_maxwell import advection_2d
from dg_maxwell import global_variables as gvar
from dg_maxwell import utils
from dg_maxwell import wave_equation_2d as w2d
from dg_maxwell import isoparam
from dg_maxwell import lagrange
from dg_maxwell import advection_2d_arbit_mesh as a2d_arbit_mesh

af.set_backend(params.backend)
af.set_device(params.device)

plt.rcParams['figure.figsize']     = 12, 7.5
plt.rcParams['lines.linewidth']    = 1.5
plt.rcParams['font.family']        = 'serif'
plt.rcParams['font.weight']        = 'bold'
plt.rcParams['font.size']          = 20  
plt.rcParams['font.sans-serif']    = 'serif'
plt.rcParams['text.usetex']        = True
plt.rcParams['axes.linewidth']     = 1.5
plt.rcParams['axes.titlesize']     = 'medium'
plt.rcParams['axes.labelsize']     = 'medium'

plt.rcParams['xtick.major.size']   = 8
plt.rcParams['xtick.minor.size']   = 4
plt.rcParams['xtick.major.pad']    = 8
plt.rcParams['xtick.minor.pad']    = 8
plt.rcParams['xtick.color']        = 'k'
plt.rcParams['xtick.labelsize']    = 'medium'
plt.rcParams['xtick.direction']    = 'in'    

plt.rcParams['ytick.major.size']   = 8
plt.rcParams['ytick.minor.size']   = 4
plt.rcParams['ytick.major.pad']    = 8
plt.rcParams['ytick.minor.pad']    = 8
plt.rcParams['ytick.color']        = 'k'
plt.rcParams['ytick.labelsize']    = 'medium'
plt.rcParams['ytick.direction']    = 'in'
plt.rcParams['text.usetex']        = True
plt.rcParams['text.latex.unicode'] = True

gmshtranslator friendly reminder: rules and actions prototypes


def node_condition(tag,x,y,z,physgroups): 
def node_action(tag,x,y,z):
def element_condition(eletag,eletype,physgrp,nodes):
def element_action(eletag,eletype,physgrp,nodes):


In [3]:
params.mesh_file = '../read_and_plot_mesh/mesh/particle_in_rectangle.msh'
advec_var = gvar.advection_variables(params.N_LGL, params.N_quad,
                                     params.x_nodes, params.N_Elements,
                                     params.c, params.total_time, params.wave,
                                     params.c_x, params.c_y, params.courant,
                                     params.mesh_file, params.total_time_2d)

/home/ubermensch/.local/anaconda3/lib/python3.6/site-packages/numpy/lib/polynomial.py:1193: FutureWarning: In the future extra properties will not be copied across when constructing one poly1d from another
  other = poly1d(other)
/home/ubermensch/.local/anaconda3/lib/python3.6/site-packages/numpy/lib/polynomial.py:1220: FutureWarning: In the future extra properties will not be copied across when constructing one poly1d from another
  other = poly1d(other)


gmshTranslator: Initializing...
gmshTranslator: Mesh has 988 nodes.
gmshTranslator: Mesh has 374 elements.
gmshTranslator: Processed 1366 lines.
gmshTranslator: There are 1 physical groups available: 
gmshTranslator:      > 0
gmshTranslator: Parsing nodes
gmshTranslator: Parsing elements
gmshTranslator: No rules for elements... skipping elements.
gmshTranslator: Parsing nodes
gmshTranslator: No rules for nodes... skipping nodes.
gmshTranslator: Parsing elements
advection_variables __init__ completed


# Vectorized Surface term developmental Stage

In [82]:
# 1. Find L_p(1) and L_p(-1)
Lp = advec_var.lagrange_coeffs
Lq = advec_var.lagrange_coeffs

Lp_1           = utils.polyval_1d(Lp, af.constant(1, d0 = 1, dtype = af.Dtype.f64))
Lp_1_slow_tile = af.moddims(af.transpose(af.tile(Lp_1, d0 = 1, d1 = params.N_LGL,
                                                 d2 = params.N_LGL)),
                            d0 = params.N_LGL * params.N_LGL, d1 = params.N_LGL)

Lp_minus_1           = utils.polyval_1d(Lp, af.constant(-1, d0 = 1, dtype = af.Dtype.f64))
Lp_minus_1_slow_tile = af.moddims(af.transpose(af.tile(Lp_minus_1, d0 = 1, d1 = params.N_LGL,
                                                       d2 = params.N_LGL)),
                                  d0 = params.N_LGL * params.N_LGL, d1 = params.N_LGL)

Lq_1 = Lp_1.copy()
Lq_1_quick_tile = af.tile(Lq_1, d0 = params.N_LGL, d1 = params.N_LGL)

Lq_minus_1 = Lp_minus_1.copy()
Lq_minus_1_quick_tile = af.tile(Lq_minus_1, d0 = params.N_LGL, d1 = params.N_LGL)

# [LOOKS FINE]

In [83]:
# 2. Find Lp(xi) and Lq(eta) [N_LGL * N_LGL, N_LGL]

Lp_xi_i = utils.polyval_1d(Lp, advec_var.xi_LGL)
Lp_xi_i_slow_tile = af.tile(af.reorder(Lp_xi_i, d0 = 2, d1 = 0, d2 = 1),
                            d0 = params.N_LGL)
Lp_xi_i_slow_tile = af.moddims(Lp_xi_i_slow_tile, d0 = params.N_LGL * params.N_LGL,
                               d1 = params.N_LGL, d2 = 1)


Lq_eta_j = utils.polyval_1d(Lq, advec_var.eta_LGL)
Lq_eta_j_quick_tile = af.tile(Lq_eta_j, d0 = params.N_LGL)

# [LOOKS FINE]

In [84]:
# 3. Find F(xi, eta)

F_xi_1_eta_j = af.constant(w2d.F_x(1), d0 = params.N_LGL * params.N_LGL, d1 = params.N_LGL)
F_xi_minus_1_eta_j = af.constant(w2d.F_x(-1), d0 = params.N_LGL * params.N_LGL, d1 = params.N_LGL)
F_xi_i_eta_1 = af.constant(w2d.F_y(1), d0 = params.N_LGL * params.N_LGL, d1 = params.N_LGL)
F_xi_i_eta_minus_1 = af.constant(w2d.F_y(-1), d0 = params.N_LGL * params.N_LGL, d1 = params.N_LGL)

# [LOOKS FINE]

In [85]:
# 4. Find isoparametric derivatives dx_dxi and dy_dxi

dx_dxi = 0.1
dy_deta = 0.1

dx_dxi_tile = af.constant(dx_dxi, d0 = params.N_LGL * params.N_LGL, d1 = params.N_LGL)
dy_deta_tile = af.constant(dy_deta, d0 = params.N_LGL * params.N_LGL, d1 = params.N_LGL)

# [LOOKS FINE]

In [91]:
# 5. Calculate the surface term intergal for the left edge

integrand_left_edge = Lp_minus_1_slow_tile * Lq_eta_j_quick_tile * F_xi_minus_1_eta_j * dy_deta_tile
integrand_left_edge = lagrange.lagrange_interpolation_u(integrand_left_edge, advec_var)
# print(integrand_left_edge.shape)

RuntimeError: In function af::dim4 getOutDims(const af::dim4&, const af::dim4&, bool)
In file src/backend/ArrayInfo.cpp:178
Invalid dimension for argument 1
Expected: ldims == rdims


# Tests related to the developmental code

In [37]:
# Different modes of tiling Lp

# Quick tiling

foo = af.range(params.N_LGL)

foo_quick_tile = af.tile(foo, d0 = params.N_LGL, d1 = params.N_LGL)
# print(foo_quick_tile)

foo_slow_tile = af.moddims(af.transpose(af.tile(foo, d0 = 1, d1 = params.N_LGL, d2 = params.N_LGL)),
                           d0 = params.N_LGL * params.N_LGL, d1 = params.N_LGL)
# print(foo_slow_tile)

In [67]:
# Quick/slow tiling for foo_1

# foo_1 = foo_quick_tile[:params.N_LGL]
foo_1 = af.transpose(af.moddims(af.range(params.N_LGL * params.N_LGL),
                                d0 = params.N_LGL, d1 = params.N_LGL))

# print(foo_1)
# Quick tile
foo_1_quick_tile = af.tile(foo_1, d0 = params.N_LGL)
# print(foo_1_quick_tile)

# slow tile
foo_1_slow_tile = af.tile(af.reorder(foo_1, d0 = 2, d1 = 0, d2 = 1), d0 = params.N_LGL)
foo_1_slow_tile = af.moddims(foo_1_slow_tile, d0 = params.N_LGL * params.N_LGL, d1 = params.N_LGL, d2 = 1)
# print(foo_1_slow_tile)

# Looks fine